# Imports

Collab specific stuff:

In [5]:
running_colab = False

In [6]:
if running_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    logs_directory = 'drive/MyDrive/summaries'
    !pip install -r "drive/MyDrive/requirements.txt"
else:
    logs_directory = 'summaries'

Imports

In [1]:
import time
import gym_2048
from tensorforce import Agent, Environment
import numpy as np
from matplotlib.pyplot import imshow
%load_ext tensorboard

# Useful code snippets

Render "beautiful" 2048 grid:

`imshow(environment._environment.environment.render(mode="rgb_array"))`

Illegal move reward:

`environment._environment.environment.set_illegal_move_reward(-10)`

Log2 reward:

`log2_reward = reward if reward <= 0 else np.log2(reward)`

Show TensorBoard graphs:

`%tensorboard --logdir summaries` (with parameter `summarizer=dict(directory='summaries')` in the `Agent.create()` method)

# Hyperparams to test

`exploration`

`learning_rate`

Negative reward for illegal moves: `environment._environment.environment.set_illegal_move_reward(-1)`

`target_sync_frequency`

`batch_size` & `update_frequency`

Constant reward / Log reward

# Custom hyper params train

Modified hyperparams :

Ex: `learning_rate = 0.01` et `batch_size = 32`

In [18]:
environment = Environment.create(
    environment='gym', level='2048-v0', max_episode_timesteps=1000
)

# Defaut hyperparams
agent = Agent.create(
    agent='dqn',
    batch_size=16, # Required by Tensorforce
    update_frequency=4, # Update frequency, TensorForce default : batch_size * 0.25
    environment=environment,
    learning_rate = 0.001, # (TensorForce default)
    discount = 0.99, # (TensorForce default)
    memory=10000,
    exploration=0.1, # (0 is the TensorForce default)
    target_sync_frequency=4, # (1 is the TensorForce default)
    summarizer=dict(
        directory=logs_directory,
        summaries=[
            "regularization-loss", 
            "loss",
            "episode-length",
            "episode-reward",
            "objective-loss",
            "reward",
            "update-return",
        ]
    )
)

max_tiles = []
scores = []
start_training_time = time.time()

for episode in range(10):
    state = environment.reset()
    terminal = False
    
    #Checking metrics while training
    state_freeze = state.copy()
    num_updates = 0
    num_moves = 0
    invalid_moves = 0
    start_episode_time = time.time()
    
    while not terminal:
        #Core
        action = agent.act(states=state)
        state, terminal, reward = environment.execute(actions=action)
        log2_reward = reward if reward <= 0 else np.log2(reward)
        num_updates += agent.observe(terminal=terminal, reward=log2_reward)

        #Number of moves
        num_moves += 1
        
        #Number of invalid moves
        if (state == state_freeze).all():
            invalid_moves += 1
        state_freeze = state.copy()
    
    # Storing score and max tile
    max_tiles.append(environment._environment.environment.Matrix.max())
    scores.append(environment._environment.environment.score)
    
    print('Episode {}: terminal = {}, updates={}, max_tile={}, valid_moves={}, invalid_moves={}, seconds={}'\
          .format(episode, terminal, num_updates, max_tiles[-1], num_moves-invalid_moves, invalid_moves, round(time.time() - start_episode_time,2)))

agent.close()
environment.close()
print("Last 100 episodes mean score: ", np.mean(scores[-10:]))
print("Max tile on last 100 episodes: ", max(max_tiles[-10:]))
print("Total training time (minutes): ", round((time.time() - start_training_time)/60,2))

Episode 0: terminal = 1, updates=89, max_tile=128, valid_moves=129, invalid_moves=232, seconds=10.44
Episode 1: terminal = 1, updates=54, max_tile=64, valid_moves=87, invalid_moves=131, seconds=1.33
Episode 2: terminal = 1, updates=43, max_tile=64, valid_moves=97, invalid_moves=76, seconds=1.49
Episode 3: terminal = 1, updates=127, max_tile=128, valid_moves=171, invalid_moves=338, seconds=3.83
Episode 4: terminal = 1, updates=115, max_tile=128, valid_moves=145, invalid_moves=316, seconds=3.66
Episode 5: terminal = 1, updates=46, max_tile=64, valid_moves=64, invalid_moves=120, seconds=1.4
Episode 6: terminal = 1, updates=179, max_tile=256, valid_moves=189, invalid_moves=528, seconds=5.34
Episode 7: terminal = 1, updates=64, max_tile=64, valid_moves=95, invalid_moves=162, seconds=1.49
Episode 8: terminal = 1, updates=87, max_tile=64, valid_moves=103, invalid_moves=246, seconds=2.95
Episode 9: terminal = 1, updates=74, max_tile=64, valid_moves=103, invalid_moves=193, seconds=2.8
Last 100 

In [13]:
%tensorboard --logdir $logs_directory

Reusing TensorBoard on port 6006 (pid 58033), started 0:09:05 ago. (Use '!kill 58033' to kill it.)

# Default hyper params

In [ ]:
environment = Environment.create(
    environment='gym', level='2048-v0', max_episode_timesteps=1000
)

# Defaut hyperparams
agent = Agent.create(
    agent='dqn',
    batch_size=16, # Required by Tensorforce
    update_frequency=4, # Update frequency, TensorForce default : batch_size * 0.25
    learning_rate = 0.001, # (TensorForce default)
    discount = 0.99, # (TensorForce default)
    memory=10000,
    exploration=0.1, # (0 is the TensorForce default)
    target_sync_frequency=4, # (1 is the TensorForce default)
)

for episode in range(1000):
    train...
    log2_reward = reward if reward <= 0 else np.log2(reward)